In [ ]:
import torch
from torch import nn
import numpy as np
from matplotlib import pyplot as plt

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def clip_and_step(allparam, optimizer, clip_r = None):
    norm_p=None
    grad_all = allparam.grad
    if clip_r is not None:
        norm_p = grad_all.norm().item()
        if norm_p > clip_r:
            grad_all.mul_(clip_r/norm_p)
    optimizer.step()
    return norm_p

def attention(P,Q,Z, activation = None):
    B= Z.shape[0]
    N = Z.shape[1]-1
    d = Z.shape[2]-1
    P_full =  torch.cat([P,torch.zeros(1,d).to(device)],dim=0)
    P_full =  torch.cat([P_full,torch.zeros(d+1,1).to(device)],dim=1)
    P_full[d,d] = 1
    Q_full = torch.cat([Q, torch.zeros(1,d).to(device)],dim=0)
    Q_full = torch.cat([Q_full, torch.zeros(d+1,1).to(device)],dim=1)
    A = torch.eye(N+1).to(device)
    A[N,N] = 0
    Attn = torch.einsum('BNi, ij, BMj -> BNM', (Z,Q_full,Z))
    if activation is not None:
        Attn = activation(Attn)
    key = torch.einsum('ij, BNj -> BNi', (P_full,Z))
    Output = torch.einsum('BNM,ML, BLi -> BNi', (Attn,A,key))
    return Output /N


class Transformer_F(nn.Module):
    def __init__(self, n_layer, n_head, d, var):
        super(Transformer_F, self).__init__()
        self.register_parameter('allparam', torch.nn.Parameter(torch.zeros(n_layer, n_head, 2, d, d)))
        with torch.no_grad():
            self.allparam.normal_(0,var)
        self.n_layer = n_layer
        self.n_head = n_head

    def forward(self, Z):
        for i in range(self.n_layer):
            Zi = Z
            residues = 0
            # the forwarad map of each layer is given by F(Z) = Z + attention(Z)
            for j in range(self.n_head):
                Pij = self.allparam[i,j,0,:,:]
                Qij = self.allparam[i,j,1,:,:]
                residues = residues + attention(Pij,Qij,Zi)
            Z = Zi + residues
        return Z

    #enforces top-left-dxd-block sparsity on p
    def zero_p(self):
        for i in range(self.n_layer):
            for j in range(self.n_head):
                with torch.no_grad():
                    self.allparam[i,j,0,:,:].zero_()

class Transformer_MLP_freeze(nn.Module):
    def __init__(self, n_layer, n_head, d, var, randomMLP):
        super(Transformer_MLP_freeze, self).__init__()
        self.register_parameter('allparam', torch.nn.Parameter(torch.zeros(n_layer, n_head, 2, d, d)))
        with torch.no_grad():
            self.allparam.normal_(0,var)
        self.n_layer = n_layer
        self.n_head = n_head
        self.randomMLP = randomMLP

    def forward(self, Z):
        Z[:,:,:-1] = self.randomMLP(Z[:,:,:-1])
        for i in range(self.n_layer):
            Zi = Z
            residues = 0
            # the forwarad map of each layer is given by F(Z) = Z + attention(Z)
            for j in range(self.n_head):
                Pij = self.allparam[i,j,0,:,:]
                Qij = self.allparam[i,j,1,:,:]
                residues = residues + attention(Pij,Qij,Zi)
            Z = Zi + residues
        return Z

class Transformer_MLP(nn.Module):
    def __init__(self, n_layer, n_head, d, var, hidden_dim):
        super(Transformer_MLP, self).__init__()
        self.register_parameter('allparam', torch.nn.Parameter(torch.zeros(n_layer, n_head, 2, d, d)))
        with torch.no_grad():
            self.allparam.normal_(0,var)
        self.n_layer = n_layer
        self.n_head = n_head
        self.mlp = nn.Sequential(nn.Linear(d, hidden_dim),nn.ReLU(),nn.Linear(hidden_dim, d))

    def forward(self, Z):
        Z[:,:,:-1] = self.mlp(Z[:,:,:-1].clone())
        for i in range(self.n_layer):
            Zi = Z
            residues = 0
            # the forwarad map of each layer is given by F(Z) = Z + attention(Z)
            for j in range(self.n_head):
                Pij = self.allparam[i,j,0,:,:]
                Qij = self.allparam[i,j,1,:,:]
                residues = residues + attention(Pij,Qij,Zi)
            Z = Zi + residues
        return Z

# evaluate the loss of model, given data (Z,y)
def in_context_loss(model, Z, y):
    N = Z.shape[1]-1
    d = Z.shape[2]-1
    output = model(Z)
    diff = output[:,N,d]+y
    loss = ((diff)**2).mean()
    return loss

def generate_data_mlp(N, B, d, randomMLP):
    # Generate random input data
    X = torch.FloatTensor(B, N, d).normal_(0, 1).to(device)
    X_test = torch.FloatTensor(B, 1, d).normal_(0, 1).to(device)

    # Additional transformations if mode is 'sphere' or 'gamma' [Similar to the existing generate_data function]

    X_MLP = randomMLP(X.view(-1, d)).view(B, N, d)
    X_test_MLP = randomMLP(X_test.view(-1, d)).view(B, 1, d)

    W = torch.FloatTensor(B, d).normal_(0,1).cuda()
    y = torch.einsum('bi,bni->bn', (W, X_MLP)).unsqueeze(2)
    y_zero = torch.zeros(B,1,1).cuda()
    y_test = torch.einsum('bi,bni->bn', (W, X_test_MLP)).squeeze(1)
    X_comb= torch.cat([X,X_test],dim=1)
    y_comb= torch.cat([y,y_zero],dim=1)
    Z= torch.cat([X_comb,y_comb],dim=2)

    return Z, y_test
# Setup
N=20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_layer = 3  # number of layers of transformer
d = 5        # dimension for the data
n_head = 1   # 1-headed attention
B = 1000     # minibatch size for sine data
var = 0.0001 # initializations scale of transformer parameter
clip_r =1


tuning_epochs = 1000
max_iters = 4000  # Number of Iterations to run
hidden_dim = 5
learning_rates = [0.001, 0.01, 0.05, 0.1]

# Function to run training
def train_transformer(optimizer_name, learning_rate, max_epochs):
    randomMLP = nn.Sequential(nn.Linear(d, hidden_dim),nn.ReLU(),nn.Linear(hidden_dim, d)).to(device)
    for para in randomMLP.parameters():
        para.requires_grad = False
    # Initialize model and optimizer

    # Instantiate model
    model = Transformer_MLP(n_layer, n_head, d, var, hidden_dim).to(device)
    #model = Transformer_MLP_freeze(n_layer, n_head, d, var, randomMLP).to(device)

    # Wrap the model for DataParallel
    if torch.cuda.device_count() > 1:
        print(f"Let's use {torch.cuda.device_count()} GPUs!")
        model = nn.DataParallel(model)

    if optimizer_name == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    elif optimizer_name == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.99))

    losses = []
    for epoch in range(max_epochs):
        # Generate data
        Z, y = generate_data_mlp(N=N, B=B, d=d, randomMLP=randomMLP)

        # Move data to the correct device
        Z, y = Z.to(device), y.to(device)

        # Training step
        optimizer.zero_grad()
        loss = in_context_loss(model, Z, y)
        loss.backward()
        norms = clip_and_step(model.allparam, optimizer, clip_r=clip_r)
        optimizer.zero_grad()

        # Record loss
        losses.append(loss.item())

        # Optionally print progress
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

    return losses

# Tuning learning rate

optimal_lr = {}
for optimizer_name in ['adam','sgd']:
    best_lr = None
    best_loss = float('inf')
    for lr in learning_rates:
        print(f"Training with {optimizer_name} optimizer, Learning Rate: {lr}")
        losses = train_transformer(optimizer_name, lr, tuning_epochs)
        avg_loss = np.mean(losses[-20:])  # Average loss over last epochs
        if avg_loss < best_loss:
            best_loss = avg_loss
            best_lr = lr
    optimal_lr[optimizer_name] = best_lr
    print(f"Optimal Learning Rate for {optimizer_name}: {best_lr}")


# Extended training with optimal learning rates
final_losses = {}
for optimizer_name, lr in optimal_lr.items():
    print(f"Extended Training with {optimizer_name} optimizer, Learning Rate: {lr}")
    losses = train_transformer(optimizer_name, lr, max_iters)
    final_losses[optimizer_name] = losses

# Plotting losses
plt.figure(figsize=(10, 6))
for optimizer_name, losses in final_losses.items():
    plt.plot(losses, label=f'{optimizer_name} LR={optimal_lr[optimizer_name]}')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.yscale('log')
plt.legend()
plt.show()

Training with adam optimizer, Learning Rate: 0.001
Epoch 0, Loss: 0.6350594758987427
Epoch 100, Loss: 0.158135786652565
Epoch 200, Loss: 0.12263938039541245
Epoch 300, Loss: 0.1388525813817978
Epoch 400, Loss: 0.08116383850574493
Epoch 500, Loss: 0.040455229580402374
Epoch 600, Loss: 0.03448481485247612
Epoch 700, Loss: 0.02776438742876053
Epoch 800, Loss: 0.024515120312571526
Epoch 900, Loss: 0.025764331221580505
Training with adam optimizer, Learning Rate: 0.01
Epoch 0, Loss: 0.5359437465667725
Epoch 100, Loss: 0.11812637001276016
Epoch 200, Loss: 0.07110867649316788
Epoch 300, Loss: 0.04054254665970802
Epoch 400, Loss: 0.03940672427415848
Epoch 500, Loss: 0.02384348027408123
Epoch 600, Loss: 0.01964680105447769
Epoch 700, Loss: 0.012951521202921867
Epoch 800, Loss: 0.010303180664777756
Epoch 900, Loss: 0.00881048571318388
Training with adam optimizer, Learning Rate: 0.05
Epoch 0, Loss: 0.48296472430229187
Epoch 100, Loss: 0.2387392669916153
Epoch 200, Loss: 0.0917467474937439
Epoch 